In [ ]:
import numpy as np
import copy

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib

import torch
import torch.nn as nn
import torch.nn.functional as F

import dtnnlib as dtnn

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (9, 8)

In [ ]:
# device = torch.device("cuda:0")
device = torch.device("cpu")

In [ ]:
class DistanceTransform_Epsilon(dtnn.DistanceTransformBase):
    
    def __init__(self, input_dim, num_centers, p=2, bias=False, epsilon=0.1, itemp=1):
        super().__init__(input_dim, num_centers, p=2)
        
        nc = num_centers
        self.scaler = nn.Parameter(torch.log(torch.ones(1, 1)*itemp))
        if epsilon is not None:
            nc += 1
        self.bias = nn.Parameter(torch.ones(1, nc)*0) if bias else None
        self.epsilon = epsilon
        
    def forward(self, x):
        dists = super().forward(x)
        
        if self.epsilon is not None:
            #################################
            dists = torch.cat([dists, torch.ones(len(x), 1).to(x)*self.epsilon], dim=1)
            #################################
        
        ## scale the dists (1 is optional)
        dists = 1-dists*torch.exp(self.scaler)
    
        if self.bias is not None: dists = dists+self.bias
        return dists

In [ ]:
class DistanceTransform_minExp_Epsilon(dtnn.DistanceTransformBase):
    
    def __init__(self, input_dim, num_centers, p=2, bias=False, epsilon=0.1, itemp=1):
        super().__init__(input_dim, num_centers, p=2)
        
        nc = num_centers
        self.scaler = nn.Parameter(torch.log(torch.ones(1, 1)*itemp))
        if epsilon is not None:
            nc += 1
        self.bias = nn.Parameter(torch.ones(1, nc)*0) if bias else None
        self.epsilon = epsilon
        
    def forward(self, x):
        dists = super().forward(x)
        
        if self.epsilon is not None:
            #################################
            dists = torch.cat([dists, torch.ones(len(x), 1).to(x)*self.epsilon], dim=1)
            #################################
        
        dists = dists-dists.min(dim=1, keepdim=True)[0]
        dists = torch.exp(-dists*self.scaler)
    
        if self.bias is not None: dists = dists+self.bias
        return dists

In [ ]:
class LocalMLP_epsilonsoftmax(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, output_dim, epsilon=1.0, itemp=1):
        super().__init__()
        
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.new_hidden_dim = 0
        self.output_dim = output_dim
        
        self.layer0 = DistanceTransform_Epsilon(self.input_dim, self.hidden_dim, bias=False, epsilon=epsilon, itemp=itemp)
        
        hdim = self.hidden_dim
        if epsilon is not None:
            hdim += 1
            
        self.softmax = nn.Softmax(dim=-1)
        self.layer1 = nn.Linear(hdim, self.output_dim)
    
        self.temp_maximum = None
        
    def forward(self, x):
        xo = self.layer0(x)

        ## drouout here is suitable for activation function (the probabilities do not add upto 1)
#         xo = F.dropout(xo, p=0.1, training=self.training)

        xo = self.softmax(xo)
        self.temp_maximum = xo.data
        
#         if self.training: ## to not use epsilon Neuron
#             self.layer1.weight.data[:,-1]*=0.
        xo = self.layer1(xo)
        return xo

In [ ]:
class LocalMLP_unNormalized(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, output_dim, epsilon=1.0, itemp=1):
        super().__init__()
        
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.new_hidden_dim = 0
        self.output_dim = output_dim
        
        self.layer0 = DistanceTransform_minExp_Epsilon(self.input_dim, self.hidden_dim, bias=False, epsilon=epsilon, itemp=itemp)
        
        hdim = self.hidden_dim
        if epsilon is not None:
            hdim += 1
            
        self.softmax = nn.Identity()
        self.layer1 = nn.Linear(hdim, self.output_dim)
    
        self.temp_maximum = None
        
    def forward(self, x):
        xo = self.layer0(x)
        xo = self.softmax(xo)
        
        self.temp_maximum = xo.data
        xo = self.layer1(xo)
        return xo

## 1. Classification

In [ ]:
def twospirals(n_points, noise=.5, angle=784):
    """
     Returns the two spirals dataset.
    """
    n = np.sqrt(np.random.rand(n_points,1)) * angle * (2*np.pi)/360
    d1x = -np.cos(n)*n + np.random.rand(n_points,1) * noise
    d1y = np.sin(n)*n + np.random.rand(n_points,1) * noise
    return (np.vstack((np.hstack((d1x,d1y)),np.hstack((-d1x,-d1y)))), 
            np.hstack((np.zeros(n_points),np.ones(n_points))))

In [ ]:
np.random.seed(1)
x, y = twospirals(300, angle=560)
x, y = x/x.max(axis=0, keepdims=True), y.reshape(-1)
xx, yy = torch.FloatTensor(x), torch.FloatTensor(y.reshape(-1,1))

x1 = xx[:,0]
x2 = xx[:,1]

%matplotlib inline
plt.figure(figsize=(5,5))
plt.scatter(x1, x2, c=y, marker='.')
# plt.xlabel("x1")
# plt.ylabel("x2")
plt.axis("equal")
# plt.savefig("./clf_toy_data.pdf")
plt.show()

In [ ]:
xx, yy = xx.to(device), yy.to(device)

## Choose Model Here

In [ ]:
criterion = nn.BCELoss()

In [ ]:
H = 15
### parameters are changed later to visualize the effects..
model = LocalMLP_epsilonsoftmax(2, H, 1, epsilon=0.1)
model = LocalMLP_unNormalized(2, H, 1, epsilon=0.1)
model

In [ ]:
model.layer0.scaler.data[0,0]

In [ ]:
# model.layer0.scaler.data[0,0] = 3. ## can change the scale here

In [ ]:
torch.manual_seed(1)
randidx = torch.randperm(len(xx))[:H]

In [ ]:
model.layer0.centers.data = xx[randidx] 

## Mapping like MLP
out = (yy[randidx].t()*2-1)*1.0

## For epsilon
if model.layer0.epsilon is not None:
    out = torch.cat([out, torch.ones(1, 1)*0.], dim=1)

model.layer1.weight.data = out
model.layer1.bias.data *= 0.

yout = torch.sigmoid(model(xx))
loss = criterion(yout, yy)
accuracy = ((yout>0.5).type(torch.float32) == yy).type(torch.float).mean()
loss.item(), accuracy.item()

In [ ]:
yout = torch.sigmoid(model(xx))
ax = plt.figure(figsize=(6,5)).add_subplot()
out = (yout.data.cpu().numpy()>0.5).astype(int)
ax.scatter(x1, x2, c=out, marker= '.', alpha=0.3)
## plot centroids
c = model.layer0.centers.data.cpu()
d = model.layer1.weight.data.cpu().t() #+ net.net[-1].bias.data.cpu()

for i in range(c.shape[0]):
    color = matplotlib.cm.tab20(i%20)
    ax.arrow(c[i,0], c[i,1], d[i,0], 0, head_width=0.15, head_length=0.1, fc=color, ec=color, linestyle=(0, (5, 10)))
    ax.scatter(c[i,0], c[i,1], color=color, marker= 'x')
    
# plt.tick_params(left = False, right = False , labelleft = False ,
#                 labelbottom = False, bottom = False)
plt.xlim(-1.9, 1.8)
plt.ylim(-2.2, 1.8)
plt.show()

In [ ]:
yout.shape, out.shape

In [ ]:
yout = model(xx)
ax = plt.figure(figsize=(6,5)).add_subplot()
out = (yout.data.cpu().numpy()>0.0).astype(int)

ax.scatter(x1, x2, c=out, marker= 'p', alpha=0.1)

ax.scatter(yout[:,0].data.numpy(), x2, c=out, marker= '.', alpha=1.0, cmap="coolwarm")
## plot centroids
c = model.layer0.centers.data.cpu()
d = model.layer1.weight.data.cpu().t() #+ net.net[-1].bias.data.cpu()

for i in range(c.shape[0]):
    color = matplotlib.cm.tab20(i%20)
    ax.arrow(c[i,0], c[i,1], d[i,0], 0, head_width=0.15, head_length=0.1, fc=color, ec=color, linestyle=(0, (5, 10)))
    ax.scatter(c[i,0], c[i,1], color=color, marker= 'x')

plt.show()

## Modify Here

In [ ]:
model.layer0.epsilon = None #### None #1. #0.3

In [ ]:
model.layer0.scaler.data[0,0] = 2.0 #### 0.5 # 2.0

#### Visualize the confidence per neuron in a grid

In [ ]:
def forward_intermediate(self, x):
    xo = self.layer0(x)
    xo = self.softmax(xo).data
    return xo

In [ ]:
xo = forward_intermediate(model, xx)

print(xo.shape)
print(xo.mean(dim=0), "\n ", xo.std(dim=0))
print(xo.min(dim=0)[0], "\n ", xo.max(dim=0)[0])

In [ ]:
num_points = 1000
X1 = np.linspace(-1.5, 1.5, num_points)*2
X2 = np.linspace(-1.5, 1.5, num_points)*2
X1, X2 = np.meshgrid(X1, X2)

XX = torch.Tensor(np.c_[X1.reshape(-1), X2.reshape(-1)]).to(device)

In [ ]:
YY = forward_intermediate(model, XX)
if model.layer0.epsilon is None:
    YY = YY.reshape(num_points, num_points, H) ## for non-epsilon
else:
    YY = YY.reshape(num_points, num_points, H+1) ## for epsilon
YY.shape

In [ ]:
max_actv = forward_intermediate(model, model.layer0.centers.data.cpu())
max_actv = max_actv.diag().numpy()

In [ ]:
!mkdir outputs/13_local_neuron_region

In [ ]:
for idx in range(xo.shape[1]):
    conf = YY[:,:,idx]
    conf = conf.data.cpu().numpy().reshape(X1.shape)
    print(conf.shape)
    
    ax = plt.figure(figsize=(8,8)).add_subplot()

    ax.scatter(x1, x2, c=out, marker= '.', alpha=0.3)

    ## plot centroids
    c = model.layer0.centers.data.cpu()
    for i in range(c.shape[0]):
        color = matplotlib.cm.tab20(i%20)
        ax.scatter(c[i,0], c[i,1], color=color, marker= 'x', s=100)
    
    try:
        ax.scatter(c[idx,0], c[idx,1], color="k", marker= 'X', s=100)
        print(f"center:",max_actv[idx],"max_grid:",conf.max(), max_actv[idx] >= conf.max())
    except:
        pass
    
    maxpt = XX[conf.argmax()]
    ax.scatter(maxpt[0], maxpt[1], color="r", marker= 'o', s=100)
    
    plt.imshow(conf, interpolation='nearest',
           extent=(X1.min(), X1.max(), X2.min(), X2.max()),
           alpha=0.6, cmap='gray',
           aspect='auto', origin='lower')
    
    LVLs = 20
    cs = ax.contour(X1, X2, conf, levels=LVLs, linestyles="None", colors="k", linewidths=1, zorder=-2)
    ax.clabel(cs, cs.levels, inline=True, fontsize=8, fmt="%1.2f")
    
    model_name = ""
    if isinstance(model, LocalMLP_unNormalized):
        model_name = "unnormalized_mindistexp"
    else:
        model_name = "dtSM"
    if model.layer0.epsilon is not None:
        model_name += f"_e{model.layer0.epsilon}"
    model_name+= f"_s{float(model.layer0.scaler.data[0,0]):.2f}"
    
    
    plt.savefig(f"./outputs/13_local_neuron_region/{model_name}_neuron_{idx}.pdf", bbox_inches="tight")
    plt.show()

In [ ]:
model_name